<a href="https://colab.research.google.com/github/antalvdb/mblm/blob/main/timbl_llm_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer
import re
import time
import argparse
import sys

In [ ]:
!pip install python3-timbl

import timbl

In [ ]:
# Global verbosity level
VERBOSITY = 3

In [ ]:
def pad_prompt(words, max_len=16):
    """Pad or trim the list of words to make it exactly `max_len` words."""
    if words is None:
        words = []  # Ensure words is a list
    if len(words) < max_len:
        words = ['_'] * (max_len - len(words)) + words
    else:
        words = words[-max_len:]
    return words

In [ ]:
def generate_text_from_server(host, port, initial_prompt, max_words=200):

    # Tokenize the initial prompt and convert tokens back to words
    initial_tokens = tokenizer.tokenize(initial_prompt)

    if initial_tokens is None:
        print("Tokenization failed; 'initial_tokens' is None.")
        initial_tokens = []

    # Prepare the initial prompt, padded or trimmed to 8 words
    prompt_words = pad_prompt(initial_tokens)

    generated_tokens = prompt_words[:]  # Store the full generated text

    try:
        # Create a socket connection
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            # Connect to the server
            client_socket.connect((host, port))

            # Receive the initial "Welcome" message from the server
            welcome_message = client_socket.recv(1024).decode('utf-8')
            #print("Received welcome message:", welcome_message)

            # Loop until max words generated or a period token is found
            for _ in range(max_words):
                next_word = None

                print(prompt_words)

                # Keep asking the server until a valid next word is received
                while not next_word:
                    # Prepare the message to send with "??" appended
                    message = "classify " + " ".join(prompt_words) + " ??\n"
                    client_socket.sendall(message.encode('utf-8'))

                    time.sleep(0.01)

                    # Receive the server response
                    data = client_socket.recv(1024).decode('utf-8')

                    print(data)

                    # Check for the CATEGORY line in the response
                    lines = data.strip().splitlines()
                    for line in lines:
                        if line.startswith("CATEGORY "):
                            # Extract the word inside curly brackets
                            match = re.search(r"\{(.*?)\}", line)
                            if match:
                                next_word = match.group(1)  # Extract the predicted word inside `{}`

                # Add the predicted word to the generated text
                generated_tokens.append(next_word)

                print(generated_tokens)

                # Shift prompt words and add the new word
                prompt_words = prompt_words[1:] + [next_word]

                # Stop if a period is generated
                #if next_word == ".":
                #    break

        # Detokenize the generated tokens
        generated_text = tokenizer.convert_tokens_to_string(generated_tokens)

        # Strip off original padding characters
        generated_text = generated_text.replace("_", "").strip()

        # Print the final generated text
        print("Generated text:", generated_text)

    except Exception as e:
        print(f"Error: {e}")


In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Example usage
#host = '135.181.158.92'  # Server's IP address or hostname
host = 'localhost'  # Server's IP address or hostname
port = 8888              # Port number on the server
initial_prompt = input("Enter your initial prompt: ")

generate_text_from_server(host, port, initial_prompt)